In [1]:
import galois
import numpy as np
from galois import GF
represent_galois = galois.GF(2**4)
galois_field = GF(2**4)
field_size = 2**4

#Parameter kode

rs = galois.ReedSolomon(15, 9)
n = rs.n
k = rs.k
t = rs.t

#Pembangkitan kunci

matriks_generator = rs.G
G = galois_field(matriks_generator)

def generate_non_singular_matrix(k):
    matrix = np.zeros((k, k), dtype=int)
    
    while np.linalg.matrix_rank(matrix) != k:
        matrix = np.random.randint(0, field_size , size=(k, k))

    return matrix

matriks_non_singular = generate_non_singular_matrix(k)
S = galois_field(matriks_non_singular)
Sinv = np.linalg.inv(S)

def generate_permutation_matrix(n):
    permutation_indices = np.random.permutation(range(n))
    permutation_matrix = np.zeros((n, n), dtype=int)
    permutation_matrix[np.arange(n), permutation_indices] = 1

    return permutation_matrix

matriks_permutasi = generate_permutation_matrix(n)
P = galois_field(matriks_permutasi)
Pinv = np.linalg.inv(P)

matrix = np.matmul(S, G)
PK = np.matmul(matrix, P)

def mceliece_key_gen(public_key_mceliece, private_key_mceliece):
    G = galois_field(matriks_generator)
    S = galois_field(matriks_non_singular)
    P = galois_field(matriks_permutasi)
    Sinv = np.linalg.inv(S)
    Pinv = np.linalg.inv(P)
    matrix = np.matmul(S, G)
    PK = np.matmul(matrix, P)
    public_key_mceliece = PK
    private_key_mceliece = S, P, Sinv, Pinv
    return public_key_mceliece, private_key_mceliece

public_key_mceliece = PK
np.savez_compressed('public_key_mceliece.npz', PK=PK)
print("kunci publik berhasil disimpan")

privat_key_mceliece = (S, P, Sinv, Pinv)
np.savez_compressed('private_key_mceliece.npz', S=S, P=P, Sinv=Sinv, Pinv=Pinv)
print("kunci privat berhasil disimpan")

def convert(message):
    alphabet = "ABCDEFGHIJKLMNOP"
    mapping = {alphabet[i]: i+0 for i in range(len(alphabet))}
    convert_m = ""

    for char in message:
        if char.upper() in mapping:
            convert_m += str(mapping[char.upper()]) + " "
        else:
            convert_m += char
            
    return convert_m.strip()

    convert_m = convert(message)

def padder(convert_m, k):
    if len(convert_m.split()) < k:
        padding = [0] * (k - len(convert_m.split()))
        convert_m += " " + " ".join(map(str, padding))
    mvec = [int(x) % field_size for x in convert_m.split()]
    symbol = galois_field(mvec)
    return symbol

    symbol = padder(convert_m, k)

#eror

field_size = 2**4
error_weight = 3
error_vector = np.zeros(n, dtype=int)
error_indices = np.random.choice(n, error_weight, replace=False)
error_values = np.random.randint(1, field_size, error_weight)
error_vector[error_indices] = error_values

e = galois_field(error_vector)

# Memeriksa apakah bobot vektor error sudah sesuai dengan parameter t
error_weight = np.count_nonzero(error_vector)
if error_weight == t:
    print("Bobot vektor error sesuai dengan parameter t.")
else:
    print("Bobot vektor error tidak sesuai dengan parameter t.")

#codeword
    c = np.matmul(symbol, PK)
    
def mceliece_encode(symbol,PK):
    e = galois_field(error_vector)
    y1 = (c + e)
    y = galois_field(y1)
    return y

    y = mceliece_encode(symbol, PK)

def convertion(string):
        field_size = 2**4
        number_list = list(map(int, string.split()))
        convert_value = []

        # Loop setiap angka dan pastikan berada dalam rentang 0 hingga 15
        for number in number_list:
            convert_numb = number % field_size
            convert_value.append(convert_numb)

        return convert_value

def mceliece_decode(y, Sinv, Pinv):
    cw = np.matmul(y, Pinv)
    d = rs.decode(cw)
    x = galois_field(d)
    mes = np.matmul(x, Sinv)
    return mes

    mes = mceliece_decode(y, Sinv, Pinv)

def unpadder(mes):
    last_zero_index = len(mes)
    for i in range(len(mes) - 1, -1, -1):
        if mes[i] != 0:
            break
        last_zero_index = i
    unpadded_m = mes[:last_zero_index]

    return unpadded_m

    unpadded_m = unpadder(mes)

def konversi(unpadded_m):
    alphabet = "ABCDEFGHIJKLMNOP"
    mapping = {i+0: alphabet[i] for i in range(len(alphabet))}
    ori_message = ""
    
    numbers = unpadded_m.split()
    for number in numbers:
        try:
            ori_message += mapping[int(number)]
        except ValueError:
            ori_message += number
    
    return ori_message

    unpadded_m = unpadder(mes)
    ori_message = konversi(" ".join(map(str, unpadded_m)))

def mceliece_encrypt(file_name, public_key_mceliece):
    k = 9 
    field_size = 2**4
    
    file_name = file_name
    public_key_mceliece = np.load(public_key_mceliece, allow_pickle=True)
    
    with open(file_name,"r") as file:
        message = file.read()
        
    block = 0
    encodeds = []
    
    convert_m = convert(message)
    symbol = padder(convert_m, 9)
    encoded = ''
    encode = mceliece_encode(symbol, public_key_mceliece)
    encode_str = " ".join(str(i) for i in encode)
    block += 1
    encodeds.append(encode_str)
    print("{0} block telah dienkripsi". format(block))
    print('ciphertext:',encode)
    fh = open("encode_"+file_name, "w")
    fh.write(encode_str)
    fh.close()
    print('string ciphertext:',encode_str)
    print("Enkripsi selesai")

def mceliece_decrypt(file_name, private_key_mceliece):
    field_size = 2**4
    
    file_name = file_name
    private_key_mceliece = np.load(private_key_mceliece, allow_pickle=True)
    
    with open(file_name,"r") as file:
        isi = file.read()
        
    encoded = ''
    decodeds = []
    block = 0
    convert_value = convertion(isi)
    galois_convert = galois_field(convert_value)
    decode = mceliece_decode(y, Sinv, Pinv)
    decode_str = " ".join(str(i) for i in decode)
    block += 1
    decodeds.append(decode_str)
    print('isi file:', isi)
    print('konversi isi dalam vektor:', galois_convert)
    print("{0} block telah didekripsi dari {1} block". format(block, block))
    print('pesan:',decode)
    fh = open("decode_"+file_name, "w")
    fh.write(ori_message)
    fh.close()
    print('pesan asli dalam vektor:',unpadded_m)
    print('original pesan:',ori_message)
    print("Dekripsi selesai")

kunci publik berhasil disimpan
kunci privat berhasil disimpan
Bobot vektor error sesuai dengan parameter t.


In [2]:
matriks_generator = rs.G
G = galois_field(matriks_generator)
print(G)

[[ 1  0  0  0  0  0  0  0  0 10  3  5 13  1  8]
 [ 0  1  0  0  0  0  0  0  0 15  1 13  7  5 13]
 [ 0  0  1  0  0  0  0  0  0 11 11 13  3 10  7]
 [ 0  0  0  1  0  0  0  0  0  3  2  3  8  4  7]
 [ 0  0  0  0  1  0  0  0  0  3 10 10  6 15  9]
 [ 0  0  0  0  0  1  0  0  0  5 11  1  5 15 11]
 [ 0  0  0  0  0  0  1  0  0  2 11 10  7 14  8]
 [ 0  0  0  0  0  0  0  1  0 15  9  5  8 15  2]
 [ 0  0  0  0  0  0  0  0  1  7  9  3 12 10 12]]


In [3]:
mceliece_key_gen("public_key_mceliece", "private_key_mceliece")

(GF([[ 6,  9,  0,  5,  7, 13, 12, 15,  6, 10,  6,  2, 10,  1, 10],
     [ 4,  1, 12,  3,  0,  2,  0,  7, 15, 12, 14, 11, 13,  6,  4],
     [ 9,  2, 15, 10,  0, 10, 14, 11,  2, 10,  8, 13, 11, 10,  9],
     [ 0,  5,  0, 14, 14,  3,  6,  2, 12,  1,  6, 14, 12, 10, 13],
     [12, 11, 12,  4,  7,  6,  4,  5,  0,  7,  1, 15, 14, 10,  5],
     [10, 11,  6,  8,  1,  9, 12, 10, 10,  0,  4, 10,  9,  8, 11],
     [10,  5, 10,  8,  5,  5,  2,  3,  2,  4,  2,  1, 10, 14, 12],
     [ 5,  0, 10, 13,  2, 11, 12, 15,  1, 12,  3, 15, 13,  8,  3],
     [ 8, 13,  3,  6, 15, 14,  7, 15, 14,  6,  9, 13, 11, 12,  4]],
    order=2^4),
 (GF([[15,  5,  2,  7,  6,  6,  6,  9, 10],
      [ 7,  3, 11,  0, 15, 14,  4,  1, 12],
      [11, 10, 13,  0,  2,  8,  9,  2, 10],
      [ 2, 14, 14, 14, 12,  6,  0,  5,  1],
      [ 5,  4, 15,  7,  0,  1, 12, 11,  7],
      [10,  8, 10,  1, 10,  4, 10, 11,  0],
      [ 3,  8,  1,  5,  2,  2, 10,  5,  4],
      [15, 13, 15,  2,  1,  3,  5,  0, 12],
      [15,  6, 13, 15, 14,  

In [4]:
with open("public_key_mceliece.npz","rb") as fh:
    byte_stream = fh.read()
    print(byte_stream)

b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x00!\x00M\xd0\xda\xa1\xb7\x00\x00\x00\x07\x01\x00\x00\x06\x00\x14\x00PK.npy\x01\x00\x10\x00\x07\x01\x00\x00\x00\x00\x00\x00\xb7\x00\x00\x00\x00\x00\x00\x00\x9d\x8d1N\x041\x10\x04\xc7c{l\x8f\xc7\xe3/8\xf3!mr\x01\x01<\x80\x0cDB@\x84V\xdc"\x82\x13\x87v\x81\x04x\x05\x1f\xc6|\x81\xce\xbaT\xea\xfe\xb9\xb9\xbb\xbe\xbd7\xf0\x01\x9f\xfd\xb0l\x8fk\xbfl\xfd\xeb}\xdf\xa7\xd6\x9fN\xeb\xdb:\xbf<\x9c\xd6\xc3\xf2\xc7\xaf\xe6\xe3\xb6\x0c\xbe=\xcf\xaf\xcb\xe8\xbb\x8b\xa9\xed\xcf\xcf\xa6\xf6\xdd\xfe\x1b\xa6\x04>\x14\xa9\xc4\x84l\xd8\x19\xb1\x80\x10\xaah.\xe4\x12V\x06\xd6\x8c\x1cK\xe6\xa1\x82\xaa%\x14C*\\$\x8b\x0b\xe4<\x04S\x95=g\x8a&\t38N1\x0f\x10\xbdG\x8b\x0e\r\xabx\xe0\x82Y\xea8\xa8%\xdaX,U\rU)\x951\xf3\x0bPK\x01\x02\x14\x03\x14\x00\x00\x00\x08\x00\x00\x00!\x00M\xd0\xda\xa1\xb7\x00\x00\x00\x07\x01\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x01\x00\x00\x00\x00PK.npyPK\x05\x06\x00\x00\x00\x00\x01\x00\x01\x004\x00\x00\x00\xef\x00\x00\x00\x00\x00'


In [5]:
with open("private_key_mceliece.npz","rb") as fh:
    byte_stream = fh.read()
    print(byte_stream)

b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x00!\x00\x96\xd7l\x00\x91\x00\x00\x00\xd1\x00\x00\x00\x05\x00\x14\x00S.npy\x01\x00\x10\x00\xd1\x00\x00\x00\x00\x00\x00\x00\x91\x00\x00\x00\x00\x00\x00\x00\x9d\x89\xbb\r\xc20\x14\x00\x9f_\xe2O^b{\x05w\x06)\re\x18\x80\x0eDCA\x85\xac\xc4\x88\x02\x11\xe4\x00\r0\x05\x0bcV\xe0\xba\xbb\xfblv\xeb\xed\x9e\xc1\x03\x9e~\x88S\x9f\xfc\xd2\xf9\xd7}\xe1[\xe7\x8fc\xba\xa5p9\x8ci\x88\xbf\xbe\n\xe7)\xe6>\x9d\xc25f\x9fu\xad\xeb\xe6\xad{\xbb\xbf!\xcbQ\n!*\x92E\r\xd6\x94\xac\xa9I\x03\xaa\n\t\x8d1\x8d\x00\xcexi%\xe4!I\x11\xa3\x92j(\x14\xe3\x88\x94\x87\xb6\xc8\n\x0e\x8d\x15\xda\x9aJi\xf1\x05PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x00!\x00\xfb!\xf8\xe6e\x00\x00\x00a\x01\x00\x00\x05\x00\x14\x00P.npy\x01\x00\x10\x00a\x01\x00\x00\x00\x00\x00\x00e\x00\x00\x00\x00\x00\x00\x00\x9b\xec\x17\xea\x1b\x10\xc9\xc8P\xc6P\xad\x9e\x92Z\x9c\\\xa4n\xa5\xa0^Sj\xa8\xae\xa3\xa0\x9e\x96_TR\x94\x98\x17\x9f_\x94\x92\n\x12wK\xcc)N\x05\x8a\x17g$\x16\xa4\x02\xf9\x1a\x86\xa6:\n\x86\xa6\x9a:\n\xb5

In [6]:
with open("contoh","r") as file:
    message = file.read()
    print(message)

ALICE


In [7]:
convert_m = convert(message)
print("konversi pesan asli:", convert_m)
symbol = padder(convert_m, k)
print('pesan vektor:', symbol)
e = galois_field(error_vector)
print('vektor eror:', e)
c = np.matmul(symbol, PK)
print('codeword:', c)
y = mceliece_encode(symbol, PK)
print('ciphertext:', y)
cw = np.matmul(y, Pinv)
print('codeword (cw):', cw)
d = rs.decode(cw)
print('decode codeword:', d)
x = galois_field(d)
print('decode codeword dalam GF(2^4):', x)
mes = mceliece_decode(y, Sinv, Pinv)
print('pesan:', mes)
unpadded_m = unpadder(mes)
print('pesan asli dalam vektor:', unpadded_m)
ori_message = konversi(" ".join(map(str, unpadded_m)))
print("Kata terdekripsi:", ori_message)

konversi pesan asli: 0 11 8 2 4
pesan vektor: [ 0 11  8  2  4  0  0  0  0]
vektor eror: [ 0  0  0  7 12  3  0  0  0  0  0  0  0  0  0]
codeword: [11  8  9 13  0  7  6  0 11 15 12 13  7  9  0]
ciphertext: [11  8  9 10 12  4  6  0 11 15 12 13  7  9  0]
codeword (cw): [ 0 10 13 12 11 12 11  8 15  9  4  0  7  9  6]
decode codeword: [ 0 13 13  0 11 12 11  8 15]
decode codeword dalam GF(2^4): [ 0 13 13  0 11 12 11  8 15]
pesan: [ 0 11  8  2  4  0  0  0  0]
pesan asli dalam vektor: [ 0 11  8  2  4]
Kata terdekripsi: ALICE


In [8]:
#proses enkripsi file

file_name = "contoh"
public_key_mceliece = np.load("public_key_mceliece.npz", allow_pickle=True)
k = 9 
field_size = 2**4
    
block = 0
encodeds = []
    
convert_m = convert(message)
symbol = padder(convert_m, 9)
encoded = ''
encode = mceliece_encode(symbol, public_key_mceliece)
encode_str = " ".join(str(i) for i in encode)
block += 1
encodeds.append(encode_str)
print("{0} block telah dienkripsi". format(block))
print('ciphertext:',encode)
fh = open("encode_"+file_name, "w")
fh.write(encode_str)
fh.close()
print('string ciphertext:',encode_str)
print("Enkripsi selesai")

1 block telah dienkripsi
ciphertext: [11  8  9 10 12  4  6  0 11 15 12 13  7  9  0]
string ciphertext: 11 8 9 10 12 4 6 0 11 15 12 13 7 9 0
Enkripsi selesai


In [9]:
mceliece_encrypt("contoh", "public_key_mceliece.npz")

1 block telah dienkripsi
ciphertext: [11  8  9 10 12  4  6  0 11 15 12 13  7  9  0]
string ciphertext: 11 8 9 10 12 4 6 0 11 15 12 13 7 9 0
Enkripsi selesai


In [10]:
#proses dekripsi
file_name = "encode_contoh"
private_key_mceliece = np.load("private_key_mceliece.npz", allow_pickle=True)

field_size = 2**4
    
with open(file_name,"r") as file:
    isi = file.read()
        
encoded = ''
decodeds = []
blok = 0
convert_value = convertion(isi)
galois_convert = galois_field(convert_value)
decode = mceliece_decode(y, Sinv, Pinv)
decode_str = " ".join(str(i) for i in decode)
blok += 1
decodeds.append(decode_str)
print('isi file:', isi)
print('konversi isi dalam vektor:', galois_convert)
print("{0} block telah didekripsi dari {1} block". format(block, block))
print('pesan:',decode)
fh = open("decode_"+file_name, "w")
fh.write(ori_message)
fh.close()
print('pesan asli dalam vektor:',unpadded_m)
print('original pesan:',ori_message)
print("Dekripsi selesai")

isi file: 11 8 9 10 12 4 6 0 11 15 12 13 7 9 0
konversi isi dalam vektor: [11  8  9 10 12  4  6  0 11 15 12 13  7  9  0]
1 block telah didekripsi dari 1 block
pesan: [ 0 11  8  2  4  0  0  0  0]
pesan asli dalam vektor: [ 0 11  8  2  4]
original pesan: ALICE
Dekripsi selesai


In [11]:
mceliece_decrypt("encode_contoh", "private_key_mceliece.npz")

isi file: 11 8 9 10 12 4 6 0 11 15 12 13 7 9 0
konversi isi dalam vektor: [11  8  9 10 12  4  6  0 11 15 12 13  7  9  0]
1 block telah didekripsi dari 1 block
pesan: [ 0 11  8  2  4  0  0  0  0]
pesan asli dalam vektor: [ 0 11  8  2  4]
original pesan: ALICE
Dekripsi selesai
